In [83]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
from nltk.corpus import stopwords
import altair as alt
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [84]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
train_df.drop('Unnamed: 0', axis=1, inplace=True)
train_df.set_index('id', inplace=True)
train_df.head()

,name,neighborhood_overview,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,monthly_revenue
id,,,,,,,,,,,,,,,,,,,,,
19792418,Home in Vancouver · ★4.75 · 1 bedroom · 1 bed ...,Everything you need is nearby. <br /><br />Hig...,57488206,Jessi,NaN,NaN,NaN,f,3,3,...,4.80,4.82,4.90,4.87,4.69,4.81,f,3,0.77,2108
1015650685503221866,Guest suite in Vancouver · ★New · 2 bedrooms ·...,NaN,139792573,Daniel,within a few hours,100%,100%,f,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,f,1,NaN,2730
35265562,Guest suite in Vancouver · ★4.85 · 2 bedrooms ...,Beautiful neighbourhood close to prosperous Ma...,265504225,Alex,within an hour,100%,98%,t,1,1,...,4.90,4.78,4.97,4.94,4.90,4.75,f,1,3.22,2254
911948980885194155,Home in Vancouver · ★5.0 · 1 bedroom · 1 bed ·...,We are located in a quiet residential neighbor...,22595056,Raymond,NaN,NaN,92%,t,1,1,...,5.00,5.00,5.00,5.00,4.86,5.00,f,1,1.28,3187
46069251,Guest suite in Vancouver · ★4.93 · 1 bedroom ·...,Kitsilano at it's best! Short walk to all the ...,65683877,Yendi,within an hour,100%,95%,t,2,3,...,4.93,4.89,4.97,4.97,4.96,4.85,f,1,2.01,3479


In [3]:
cols = train_df.columns
print(cols)

Index(['name', 'neighborhood_overview', 'host_id', 'host_name',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'neighbourhood', 'neighbourhood_cleansed', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count', 'reviews_per_month',
       'monthly_revenue'],
      dtype='object')


In [266]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5352 entries, 19792418 to 19298482
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      5352 non-null   int64  
 1   name                            5352 non-null   object 
 2   neighborhood_overview           3476 non-null   object 
 3   host_id                         5352 non-null   int64  
 4   host_name                       5352 non-null   object 
 5   host_response_time              4263 non-null   object 
 6   host_response_rate              4263 non-null   object 
 7   host_acceptance_rate            4620 non-null   object 
 8   host_is_superhost               5315 non-null   object 
 9   host_listings_count             5352 non-null   int64  
 10  host_total_listings_count       5352 non-null   int64  
 11  neighbourhood                   3476 non-null   object 
 12  neighbourhood_cleansed      

| Feature | Transformation | Explanation
| --- | ----------- | ----- |
| name | Custom transformation to create new columns | Extract features such as rating, Number of beds and baths, if bath is private/shared
| neighborhood_overview | Create embeddings - try to learn features | Create embeddings for this feature (maybe bert or some other). There are 1876 missing values - need to handle them. Use TfidfVectorizer with maybe 100 max features to get key terms. `df['cleaned_neighborhood_overview'].fillna('no description available', inplace=True)` |
| host_id | `drop` |  All are mostly unique, so probably useless
| host_name | `drop` |  All are mostly unique, so probably useless
| host_response_time | `ordinal/one-hot` | Categorical column. There are 1089 missing values
| host_response_rate | `Log Scale` | Numerical column. There are 1089 missing values
| host_acceptance_rate | `Log Scale` | Numerical column. There are 732 missing values
| host_is_superhost | `one-hot` | Numerical column. There are few missing values. Values are f and t. f means 0 and t means 1. Fill nans with 0
| host_listings_count | `Log Scale` | Numerical column.
| host_total_listings_count | `Log Scale` | Numerical column.
| neighbourhood | `drop` | It's just about which city. Drop it
| neighbourhood_cleansed | `one-hot` | Categorical
| latitude | `drop` | Just about the location. Probably drop it
| longitude | `drop` | Just about the location. Probably drop it
| property_type | `one-hot` | Probably categorical. But there are a lot of categories. Around 46
| room_type | `one-hot` | Categorical. 
| accommodates | `ordinal/one-hot` | Categorical.
| bathrooms | `drop` and then `NumericalScale` | Initial column has all NaNs. Populate this from `name` column.
| bedrooms | `drop` and then `NumericalScale` | Initial column has all NaNs. Populate this from `name` column.
| beds | `Numerical Scale` | Numerical. Fill NaNs with 0
| amenities | `drop` | It's all NaNs. But can maybe use `neighborhood_overview` column to extract keywords
| price | `drop` | Clean this column. They are all strings and are in this format. '$132.00'. And mostly less than $1000, around 40 rows that are above $1000. Drop the rows that are too high |
| minimum_nights | `Feature Engineer` | Can be used to create a new feature. Muliply with price to get a minimum monthly revenue column that is directly correlated to the target column - means that the revenue must be atleast as much as this new column `df['minimum_monthly_revenue'] = df['minimum_nights'] * df['price'] * 30` |
| maximum_nights | `Feature Engineer` | Can be used to create a new feature. Muliply with price to get a maximum monthly revenue column that is directly correlated to the target column - means that the revenue must be at most as much as this new column. `df['maximum_monthly_revenue'] = df['maximum_nights'] * df['price'] * 30` |
| minimum_nights_avg_ntm | `Numerical Scaler` | Use as is for now
| maximum_nights_avg_ntm | `Numerical Scaler` | Use as is for now
| availability_30, availability_60, availability_90, availability_365 | `Feature Engineer` | Create a new col Availability ratio for each - You can calculate the proportion of available days out of total days to assess how “available” the listing is relative to demand. ratios for each. Also can then create revenue related columns using the newly created ratio column. For example `df['monthly_revenue_60'] = df['availability_ratio_60'] * df['price'] * 30 `. This highly correlates to the monthly revenue which is the target column |
| number_of_reviews | `Numerical Scaler` | Use as is for now
| number_of_reviews_ltm | `Numerical Scaler` | Use as is for now
| review_scores_rating | `Numerical Scaler` | Use as is for now
| review_scores_accuracy | `Numerical Scaler` | Use as is for now
| review_scores_cleanliness | `Numerical Scaler` | Use as is for now
| review_scores_checkin | `Numerical Scaler` | Use as is for now
| review_scores_communication | `Numerical Scaler` | Use as is for now
| review_scores_location | `Numerical Scaler` | Use as is for now
| review_scores_value | `Numerical Scaler` | Use as is for now
| instant_bookable | `onehot` | Need to convert to f is 0 and t is 1
| calculated_host_listings_count | `Log Scale` | Numerical column
| reviews_per_month | `Scale` | Numerical column


| monthly_revenue | `Target` | Target column - need to predict


- Drop Features - `latitude`, `longitude`, `host_id` (maybe `host_name` as well? not sure), `neighbourhood`, `amenities`

- Categorical Features - `host_response_time`, `neighbourhood_cleansed`, `property_type`, `room_type`

- Ordinal - Maybe `host_response_rate`, `host_acceptance_rate`, `accommodates`

- Binary Features - `host_is_superhost`

Not sure what to do - `host_listings_count`, `host_total_listings_count`


In [70]:
train_df[train_df['host_id'] == 57488206]

,name,neighborhood_overview,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,monthly_revenue
id,,,,,,,,,,,,,,,,,,,,,
19792418,Home in Vancouver · ★4.75 · 1 bedroom · 1 bed ...,Everything you need is nearby. <br /><br />Hig...,57488206,Jessi,NaN,NaN,NaN,f,3,3,...,4.80,4.82,4.9,4.87,4.69,4.81,f,3,0.77,2108
24948099,Home in Vancouver · 1 bedroom · 1 bed · 1 shar...,Everything you need is nearby. <br /><br />We ...,57488206,Jessi,NaN,NaN,NaN,f,3,3,...,5.00,5.00,5.0,5.00,5.00,5.00,f,3,0.02,500
28390229,Home in Vancouver · ★4.33 · 2 bedrooms · 2 bed...,Everything you need is nearby. <br /><br />Hig...,57488206,Jessi,NaN,NaN,NaN,f,3,3,...,4.67,4.00,5.0,5.00,5.00,4.67,f,3,0.05,3420


In [66]:
# train_df[train_df['host_is_superhost'] == 30].iloc[0]
train_df['host_is_superhost'].value_counts()

host_is_superhost
f    3117
t    2198
Name: count, dtype: int64

In [51]:
train_df['price'] = train_df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
low_price_data = train_df[train_df['price'] < 1000]
high_price_data = train_df[train_df['price'] > 1000]

# Scatter plot of high-priced listings
scatter = alt.Chart(low_price_data).mark_bar().encode(
    x='price:Q',  # Price on x-axis
    y='count():Q',  # Count (for visualization purpose, we use count here)
).properties(
    title='Listings with Price Above $1000',
    width=400,
    height=300
)

scatter

alt.Chart(...)

In [80]:
heatmap = alt.Chart(low_price_data).mark_rect().encode(
    x=alt.X('price:Q', bin=alt.Bin(maxbins=30)),  # Binned price values
    y=alt.Y('monthly_revenue:Q', bin=alt.Bin(maxbins=30)),  # Binned monthly revenue values
    color='count():Q',  # Color by the count of points in each bin  # Display binned price, monthly revenue, and count
).properties(
    title='Heatmap of Price vs Monthly Revenue',
    width=300,
    height=200
)

heatmap

alt.Chart(...)

In [256]:
train_df.drop(train_df.columns[0], axis=1, inplace=True)
train_df.drop('bathrooms', axis=1, inplace=True)
train_df.drop('bedrooms', axis=1, inplace=True)

In [258]:
missing_values = train_df.isnull().sum()
print(missing_values[missing_values > 0].sort_values(ascending=False))  # Columns with missing data

neighborhood_overview          1876
neighbourhood                  1876
host_response_time             1089
host_response_rate             1089
reviews_per_month               906
review_scores_checkin           901
review_scores_location          901
review_scores_value             901
review_scores_rating            900
review_scores_accuracy          900
review_scores_cleanliness       900
review_scores_communication     900
host_acceptance_rate            732
price                           665
host_is_superhost                37
beds                             36
dtype: int64


In [259]:
train_df.columns

Index(['name', 'neighborhood_overview', 'host_id', 'host_name',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'neighbourhood', 'neighbourhood_cleansed', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates', 'beds', 'amenities',
       'price', 'minimum_nights', 'maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count', 'reviews_per_month',
       'monthly_revenue'],
      dtype='object')

`host_is_superhost` - Values are f and t
`instant_bookable` - Values are f and t

In [252]:
train_df = train_df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)

In [99]:
# for idx, i in enumerate(train_df['name']):
for idx, i in enumerate(train_df['name']):
    print(i)
    if idx == 15:
        break

Home in Vancouver · ★4.75 · 1 bedroom · 1 bed · 1 bath
Guest suite in Vancouver · ★New · 2 bedrooms · 2 beds · 1 bath
Guest suite in Vancouver · ★4.85 · 2 bedrooms · 3 beds · 1 bath
Home in Vancouver · ★5.0 · 1 bedroom · 1 bed · 1 bath
Guest suite in Vancouver · ★4.93 · 1 bedroom · 2 beds · 1 bath
Condo in Vancouver · ★4.88 · Studio · 1 bed · 1 bath
Condo in Vancouver · ★4.76 · 2 bedrooms · 2 beds · 1 bath
Condo in Vancouver · ★5.0 · 2 bedrooms · 2 beds · 2 baths
Rental unit in Vancouver · ★New · Studio · 1 bed · 1 bath
Townhouse in Vancouver · ★4.94 · 2 bedrooms · 2 beds · 1 bath
Home in Vancouver · ★New · 1 bedroom · 1 bed · 1 bath
Guest suite in Vancouver · 2 bedrooms · 2 beds · 1 private bath
Home in Vancouver · ★4.97 · 1 bedroom · 2 beds · 1 bath
Home in Vancouver · ★4.67 · 1 bedroom · 1 bed · 1 bath
Townhouse in Vancouver · ★4.89 · 1 bedroom · 1 bed · 1 private bath
Condo in Vancouver · ★5.0 · 1 bedroom · 2 beds · 1 bath


In [111]:
a = train_df["name"].str.split("·").iloc[0]
a[1].strip()[0] == '★'

True

In [115]:
#The code below extracts features and creates new columns from the column 'name'
# Extract overall rating
def extract_rating(parts):
    for part in parts:
        if '★' in part:
            try:
                return float(part.replace('★', '').strip())
            except ValueError:
                continue
    return 0  # Return None if no rating is found

# Check if property is new
def is_property_new(parts):
    for part in parts:
        if 'new' in part.lower():
            return 1
    return 0

# Extract bedrooms
def extract_bedrooms(parts):
    for part in parts:
        if 'Studio' in part:
            return 0  # Studio = 0 bedrooms
        elif 'bedroom' in part:
            try:
                return int(part.split()[0])
            except ValueError:
                continue
    return 0

# Extract beds
def extract_beds(parts):
    for part in parts:
        if 'bed' in part:
            try:
                return int(part.split()[0])
            except ValueError:
                continue
    return 0

# Extract baths
def extract_baths(parts):
    for part in parts:
        if 'half-bath' in part.lower():
            return 0.5
        if 'bath' in part.lower():
            try:
                return float(part.split()[0])
            except ValueError:
                continue
    return 0  # Return 0 if no bath info is found

# Update private/shared bath flags to handle "Half-bath"
def is_private_bath(parts):
    for part in parts:
        if 'private' in part.lower() and 'bath' in part.lower():
            return 1
    return 0

def is_shared_bath(parts):
    for part in parts:
        if 'shared' in part.lower() and 'bath' in part.lower():
            return 1
    return 0

# Apply functions
train_df["split_parts"] = train_df["name"].str.split("·")
train_df["bedrooms"] = train_df["split_parts"].apply(extract_bedrooms)
train_df["beds"] = train_df["split_parts"].apply(extract_beds)
train_df["baths"] = train_df["split_parts"].apply(extract_baths)
train_df["is_bath_private"] = train_df["split_parts"].apply(is_private_bath).astype(int)
train_df["is_bath_shared"] = train_df["split_parts"].apply(is_shared_bath).astype(int)
train_df["overall_rating"] = train_df["split_parts"].apply(extract_rating)
train_df["is_new_property"] = train_df["split_parts"].apply(is_property_new).astype(int)

# Drop the temporary split column
train_df.drop('split_parts', axis=1, inplace=True)

# Display results
train_df.head()

,name,neighborhood_overview,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,instant_bookable,calculated_host_listings_count,reviews_per_month,monthly_revenue,cleaned_neighborhood_overview,baths,is_bath_private,is_bath_shared,overall_rating,is_new_property
id,,,,,,,,,,,,,,,,,,,,,
19792418,Home in Vancouver · ★4.75 · 1 bedroom · 1 bed ...,Everything you need is nearby. <br /><br />Hig...,57488206,Jessi,NaN,NaN,NaN,f,3,3,...,f,3,0.77,2108,everything you need is nearby highlight walkin...,1.0,0,0,4.75,0
1015650685503221866,Guest suite in Vancouver · ★New · 2 bedrooms ·...,NaN,139792573,Daniel,within a few hours,100%,100%,f,1,4,...,f,1,NaN,2730,,1.0,0,0,0.00,1
35265562,Guest suite in Vancouver · ★4.85 · 2 bedrooms ...,Beautiful neighbourhood close to prosperous Ma...,265504225,Alex,within an hour,100%,98%,t,1,1,...,f,1,3.22,2254,beautiful neighbourhood close to prosperous ma...,1.0,0,0,4.85,0
911948980885194155,Home in Vancouver · ★5.0 · 1 bedroom · 1 bed ·...,We are located in a quiet residential neighbor...,22595056,Raymond,NaN,NaN,92%,t,1,1,...,f,1,1.28,3187,we are located in a quiet residential neighbor...,1.0,0,0,5.00,0
46069251,Guest suite in Vancouver · ★4.93 · 1 bedroom ·...,Kitsilano at it's best! Short walk to all the ...,65683877,Yendi,within an hour,100%,95%,t,2,3,...,f,1,2.01,3479,kitsilano at its best short walk to all the lo...,1.0,0,0,4.93,0


In [ ]:
drop_features = ['host_id', 'host_name', 'latitude', 'longitude', 'amenities', 'bathrooms', 'bedrooms', 'host_listings_count', 'host_total_listings_count', 'neighbourhood']
cat_features = ['host_response_time', 'host_is_superhost', 'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'beds', 
                'instant_bookable', 'calculated_host_listings_count', 'is_bath_private', 'is_bath_shared']
num_features = ['price', 'minimum_nights', 'maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_30', 'availability_60', 
                'availability_90', 'availability_365', 'number_of_reviews', 'number_of_reviews_ltm', 'review_scores_rating','review_scores_accuracy', 
                'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month',
                'bedrooms', 'beds', 'baths']

#Need to scale - host_response_rate, host_acceptance_rate
#Log scale - host_listings_count

In [ ]:
def clean_text(text):
    if pd.isna(text):  # Handle missing values
        return ''
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_df['cleaned_neighborhood_overview'] = train_df['neighborhood_overview'].apply(clean_text)

tfidf = TfidfVectorizer(max_features=100, stop_words='english')  # Limit to top 100 features for simplicity
tfidf_matrix = tfidf.fit_transform(train_df['cleaned_neighborhood_overview'])

# Convert TF-IDF matrix to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

/var/folders/r9/kntgjpl16bj6tzwn1k22w0g40000gn/T/ipykernel_84008/2197921103.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
/var/folders/r9/kntgjpl16bj6tzwn1k22w0g40000gn/T/ipykernel_84008/2197921103.py:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [97]:
tfidf_df.columns

Index(['access', 'airport', 'area', 'arena', 'art', 'attractions', 'away',
       'bars', 'bc', 'beach', 'beaches', 'beautiful', 'best', 'bike', 'block',
       'blocks', 'bus', 'cafes', 'canada', 'center', 'central', 'centre',
       'city', 'close', 'coffee', 'commercial', 'community', 'distance',
       'district', 'downtown', 'drive', 'east', 'easy', 'end', 'english',
       'enjoy', 'explore', 'food', 'gastown', 'granville', 'great', 'grocery',
       'heart', 'home', 'house', 'including', 'island', 'just', 'kitsilano',
       'known', 'local', 'located', 'location', 'main', 'market', 'min',
       'mins', 'minute', 'minutes', 'nearby', 'neighborhood', 'neighbourhood',
       'north', 'park', 'parks', 'place', 'popular', 'public', 'quiet',
       'residential', 'restaurants', 'right', 'robson', 'rogers', 'safe',
       'seawall', 'shopping', 'shops', 'short', 'skytrain', 'st', 'stanley',
       'station', 'steps', 'store', 'stores', 'street', 'streets', 'th',
       'transit', 'tr

In [192]:
# train_df[train_df['beds'].isna()] == True
nan_rows = train_df[train_df[['bedrooms']].isna().any(axis=1)]
# nan_rows.head()
for idx, i in enumerate(nan_rows['name']):
# for idx, i in enumerate(train_df['neighborhood_overview']):
    print(i)
    # print(i.split('·'))
    # print(re.search(r"(\d+(\.\d+)?)\s+baths?", i, re.IGNORECASE))
    # if idx == 5:
    #     break

# train_df['beds'].hasnans
# train_df['baths'].hasnans
